In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

import warnings
warnings.simplefilter("ignore", ResourceWarning)

# Vectrix Demo 👨🏻‍💻
This notebook demonstrates the functions for importing data from various sources. 
Loading it into a VectorStore, and then using it to answer questions with a Retrieval Augemented Reasoning  🦜🔗 LangGraph.

## Creating a new project



In [2]:
from vectrix.db import DB
db = DB()
print(db.list_projects())

[]


In [ ]:
print(db.get_collection_metdata("Vectrix"))

In [3]:
db.create_project("Vectrix", description="Demo project for Vectrix")

1

In [ ]:
db.remove_project("Vectrix")

## Importing Data
### 1. From a URL 🔗

**Web Crawling and Data Extraction Example**<>

This cell demonstrates the process of crawling a website, chunking the extracted content, and performing Named Entity Recognition (NER) using the Vectrix library.

#### Steps:

1. **Web Crawling**: 
   - Uses `vectrix.Crawler` to extract pages from "https://vectrix.ai"
   - Limits the crawl to a maximum of 20 pages

2. **Content Chunking**:
   - Utilizes `vectrix.Webchunker` to break down the extracted content
   - Chunks are created with a size of 500 characters and an overlap of 50 characters

3. **Named Entity Recognition**:
   - Employs `vectrix.Extract` with the 'Replicate' model
   - Uses the 'meta/meta-llama-3-70b-instruct' model for entity extraction

This example showcases a typical workflow for web data extraction and processing using the Vectrix library.

In [4]:
from vectrix.importers import WebScraper

scraper = WebScraper("https://www.vectrix.ai/", "Vectrix")
all_links = scraper.get_all_links()

Set/updated scrape status for base URL: https://www.vectrix.ai/


2024-08-20 16:46:56,737 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-20 16:46:56,737 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-20 16:46:56,737 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-20 16:46:57,156 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-20 16:46:57,156 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-20 16:46:57,156 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-20 16:46:57,268 - courlan.urlstore - WARNING - Discarding URL: /blog-post/understanding-large-and-small-language-models-key-differences-and-applications
2024-08-20 16:46:57,268 - courlan.urlstore - WARNING - Discarding URL: /blog-post/understanding

Added links to confirm for base URL: https://www.vectrix.ai/
Set/updated scrape status for base URL: https://www.vectrix.ai/


In [ ]:
web_pages = scraper.get_all_pages(all_links)
len(web_pages)

In [ ]:
from vectrix.importers import chunk_content
chunked_webpages = chunk_content(web_pages)


print(f"Before chunking we had {len(web_pages)} and after chunking {len(chunked_webpages)}")

## Adding the data to a VectorStore

In [ ]:
db.add_documents(chunked_webpages, "Vectrix")

In [ ]:
print(db.similarity_search("Vectrix", "When was Vectrix founded?", 1))

### From OneDrive ☁
We can also connect with a OneDrive folder and download the information from there. After this process we can add the information to our Vector store in the same way. This way we can extract various kinds of documents like PDF, World, PowerPoint ...

Downloads files from OneDrive, processes them with Unstructured, and stores in Weaviate.

Steps:
1. Connect to OneDrive using Azure credentials
2. List and download all files
3. Process files with Unstructured importer
4. Add processed documents to Weaviate vector store
5. Close OneDrive connection

Requirements:
- vectrix library (OneDrive connector, Unstructured importer)
- Weaviate client
- Azure credentials as environment variables (AZURE_CLIENT_ID, AZURE_CLIENT_SECRET)

Note: Close OneDrive connection before uploading to vector store.

In [ ]:
from vectrix.connectors.onedrive import OneDrive
from vectrix.importers.unstructured import Unstructured

drive = OneDrive(azure_client_id = os.environ.get('AZURE_CLIENT_ID'), azure_client_secret = os.environ.get('AZURE_CLIENT_SECRET'))

drive.list_files()

In [ ]:
# Download all files in the drive, returns a list of downloaded files
downloaded_files = drive.download_files()
print(downloaded_files)

# Process teh documents using unstructured (we can't load RAW PDF files into a Vector store)
importer = Unstructured()
documents = importer.process_files(downloaded_files)

# Add the documents to the Vector store
weaviate = Weaviate()
weaviate.set_colleciton('Vectrix')
weaviate.add_data(documents)

# Close the drive, do this BEFORE you want to upload the files to a vector store
drive.close()